# Сбор данных с сайта [quotes](https://quotes.toscrape.com/)

In [5]:
import requests
from bs4 import BeautifulSoup
import json
from tqdm import tqdm

In [7]:
# Функция для получения информации об авторе
def get_author_info(author_url):
    response = requests.get(author_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Извлечение информации об авторе
    birth_date = soup.find('span', class_='author-born-date').get_text()
    birth_place = soup.find('span', class_='author-born-location').get_text()
    description = soup.find('div', class_='author-description').get_text(strip=True)

    return {
        "birth_date": birth_date,
        "birth_place": birth_place,
        "description": description
    }

# URL сайта
url = "https://quotes.toscrape.com/"

# Получение HTML-кода страницы
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Сбор данных
quotes = []
for quote in tqdm(soup.find_all("div", class_="quote")):
    text = quote.find("span", class_="text").get_text()
    author = quote.find("small", class_="author").get_text()

    # Получение ссылки на страницу автора
    author_link = quote.find("small", class_="author").find_next_sibling("a")['href']
    author_url = f"https://quotes.toscrape.com{author_link}"

    # Получение информации об авторе
    author_info = get_author_info(author_url)

    quotes.append({
        "text": text,
        "author": author,
        "author_info": author_info
    })

# Сохранение данных в JSON файл
with open("quotes_with_authors.json", "w", encoding="utf-8") as json_file:
    json.dump(quotes, json_file, ensure_ascii=False, indent=4)

print("\n\nДанные успешно собраны и сохранены в quotes_with_authors.json")

100%|██████████| 10/10 [00:03<00:00,  3.15it/s]



Данные успешно собраны и сохранены в quotes_with_authors.json
